In [3]:
import pandas as pd
import pickle
from scipy.stats import poisson


In [6]:
dict_table = pickle.load(open('../data/tables_group', 'rb'))
df_historical_data = pd.read_csv('../data/clean_fifa_worldcup_data_historical.csv')
df_fixture = pd.read_csv('../data/clean_fifa_worldcup_fixture.csv')

### CALCULATE TEAM STRENGTH

In [7]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
859,Russia,Croatia,2018,2,2
860,France,Belgium,2018,1,0
861,Croatia,England,2018,2,1
862,Belgium,England,2018,2,0


In [8]:
# split df in df_home and df_away
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]

In [10]:
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]
df_away

,AwayTeam,HomeGoals,AwayGoals
0,Mexico,4,1
1,France,1,0
2,Mexico,3,0
3,France,1,0
4,Mexico,6,3
...,...,...,...
859,Croatia,2,2
860,Belgium,1,0
861,England,2,1
862,England,2,0


In [11]:
# rename columns
df_home= df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

In [12]:
# concatenate df_home and df_away, group them by team and obtain average
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.717949,1.166667
Australia,0.812500,1.937500
Austria,1.576923,1.692308
...,...,...
Uruguay,1.603774,1.339623
Wales,0.800000,0.800000
West Germany,2.050847,1.254237


### CREATE FUNCTION TO PREDICT POINTS

In [13]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lambda_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, 'GoalsConceded']
        lambda_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, 'GoalsConceded']
        prob_home, prob_away, prob_draw = 0,0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lambda_home) * poisson.pmf(y, lambda_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p

        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

### TEST FUNCTION


In [16]:
# Test with matches: Argentina - Mexico, England - United States, Qatar(H) - Ecuador
predict_points('Argentina', 'Mexico')
predict_points('England', 'United States')
predict_points('Qatar (H)', 'Ecuador')

(0, 0)

### GROUP STAGES


In [17]:
# Dividing the match into groups, round of 16 and quarterfinals
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [18]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [19]:
for group in dict_table:
    print(dict_table[group]['Team'].values)

['Qatar (H)' 'Ecuador' 'Senegal' 'Netherlands']
['England' 'Iran' 'United States' 'Wales']
['Argentina' 'Saudi Arabia' 'Mexico' 'Poland']
['France' 'Australia' 'Denmark' 'Tunisia']
['Spain' 'Costa Rica' 'Germany' 'Japan']
['Belgium' 'Canada' 'Morocco' 'Croatia']
['Brazil' 'Serbia' 'Switzerland' 'Cameroon']
['Portugal' 'Ghana' 'Uruguay' 'South Korea']


In [20]:
# predict group stage matches and update the tables for each group
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)


C:\Users\Diana Laura\AppData\Local\Temp\ipykernel_4476\1050066887.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.87343108]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Diana Laura\AppData\Local\Temp\ipykernel_4476\1050066887.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39716107]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Diana Laura\AppData\Local\Temp\ipykernel_4476\1050066887.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.78567341]' has dtype incompati

### ROUND OF 16


In [22]:
# round of 16: df_fixture_knockout
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [23]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [24]:
# Update the round of 16 fixture with first place (winner group) and second place (runner-up group)
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}': group_winner, f'Runners-up {group}': runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout


,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [27]:
# create a function to get the winner
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away= predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner

    return df_fixture_updated

In [31]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Croatia
55,Portugal,Match 56,Switzerland,2022,Portugal


### QUARTER FINALS


In [32]:
# create a function to update the tables

def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}': winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [33]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Croatia,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [34]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Croatia,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


### SEMIFINAL

In [35]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [36]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


### FINAL

In [37]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [38]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil
